### **CS551G - Data Mining and Visualisation**

---
<br>

Assessment I (Task 2): Time Series Classification of User Movement Data

Name: Avinash Vinayak Bagul.
ID: 51987820.
Topic: Assessment 1, Task 2

Importing Libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
import os
import json

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_recall_fscore_support,
    accuracy_score,
)
from sklearn.model_selection import StratifiedKFold
from os import listdir
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Uploading Dataset....

In [2]:
from google.colab import files
files.upload()

Saving .DS_Store to .DS_Store
Saving Ambient_Living_1.csv to Ambient_Living_1.csv
Saving Ambient_Living_2.csv to Ambient_Living_2.csv
Saving Ambient_Living_3.csv to Ambient_Living_3.csv
Saving Ambient_Living_4.csv to Ambient_Living_4.csv
Saving Ambient_Living_5.csv to Ambient_Living_5.csv
Saving Ambient_Living_6.csv to Ambient_Living_6.csv
Saving Ambient_Living_7.csv to Ambient_Living_7.csv
Saving Ambient_Living_8.csv to Ambient_Living_8.csv
Saving Ambient_Living_9.csv to Ambient_Living_9.csv
Saving Ambient_Living_10.csv to Ambient_Living_10.csv
Saving Ambient_Living_11.csv to Ambient_Living_11.csv
Saving Ambient_Living_12.csv to Ambient_Living_12.csv
Saving Ambient_Living_13.csv to Ambient_Living_13.csv
Saving Ambient_Living_14.csv to Ambient_Living_14.csv
Saving Ambient_Living_15.csv to Ambient_Living_15.csv
Saving Ambient_Living_16.csv to Ambient_Living_16.csv
Saving Ambient_Living_17.csv to Ambient_Living_17.csv
Saving Ambient_Living_18.csv to Ambient_Living_18.csv
Saving Ambient_L

{'.DS_Store': b'\x00\x00\x00\x01Bud1\x00\x00P\x00\x00\x00\x08\x00\x00\x00P\x00\x00\x00\x10\x0c\x00\x00\x01\x08\x00\x00 \x0c\x00\x000\x0c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\xd9\x00\x00\x00\x05\x00\x00\x10\x00\x00b\x00i\x00e\x00n\x00t\x00_\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

**Dataset Description**

**Ambient Living Wireless Sensor Network Dataset:**

---


Dataset contains data from wireless sensor network deployed in real world office environments. Ambient Living dataset is given in 216 different .csv files, to read the data, all the files were uploaded on colab. Each file contain data given by 4 sensors. Given data files are of different length. There is no missing data in the dataset. All the sensor data have float datatype. Uploaded files are being read using pandas read_csv function and values are appended in a list. Shape of the appended list is (8080, 4).


**Ambient Living Target Dataset:**

---


Target labels are of int datatype.
This dataset contains the class labels (1,-1)with the data shape (216, 1). There are 111 examples of (-1) class and 105 examples of (1) class.


**Reading Dataset....**

In [3]:
path ='Ambient_Living_'
sequences = list()
dt = list()
for i in range(1,217):
    file_path = path + str(i) + '.csv'
    print(file_path)
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)
    dt.append(df)

data = pd.concat(dt, axis=0) 
targets = pd.read_csv('Ambient_Living_target.csv', index_col="#sequence_ID")
targets.index.name = None
targets.columns = ["label"]

Ambient_Living_1.csv
Ambient_Living_2.csv
Ambient_Living_3.csv
Ambient_Living_4.csv
Ambient_Living_5.csv
Ambient_Living_6.csv
Ambient_Living_7.csv
Ambient_Living_8.csv
Ambient_Living_9.csv
Ambient_Living_10.csv
Ambient_Living_11.csv
Ambient_Living_12.csv
Ambient_Living_13.csv
Ambient_Living_14.csv
Ambient_Living_15.csv
Ambient_Living_16.csv
Ambient_Living_17.csv
Ambient_Living_18.csv
Ambient_Living_19.csv
Ambient_Living_20.csv
Ambient_Living_21.csv
Ambient_Living_22.csv
Ambient_Living_23.csv
Ambient_Living_24.csv
Ambient_Living_25.csv
Ambient_Living_26.csv
Ambient_Living_27.csv
Ambient_Living_28.csv
Ambient_Living_29.csv
Ambient_Living_30.csv
Ambient_Living_31.csv
Ambient_Living_32.csv
Ambient_Living_33.csv
Ambient_Living_34.csv
Ambient_Living_35.csv
Ambient_Living_36.csv
Ambient_Living_37.csv
Ambient_Living_38.csv
Ambient_Living_39.csv
Ambient_Living_40.csv
Ambient_Living_41.csv
Ambient_Living_42.csv
Ambient_Living_43.csv
Ambient_Living_44.csv
Ambient_Living_45.csv
Ambient_Living_46.c

In [4]:
print(data.head())
print("\n",data.shape)

   Sensor 1  Sensor 2  Sensor 3  Sensor 4
0  -0.90476     -0.48   0.28571      0.30
1  -0.57143     -0.32   0.14286      0.30
2  -0.38095     -0.28  -0.14286      0.35
3  -0.28571     -0.20  -0.47619      0.35
4  -0.14286     -0.20   0.14286     -0.20

 (8080, 4)


In [5]:
print(targets.head())
print("\n ",targets.shape)

   label
1      1
2      1
3      1
4      1
5      1

  (216, 1)


***Checking for the missing data in both: ambient data and target data***

In [6]:
print("Ambient sensor dataset: \n\n",data.isnull())

# no missing values in the ambient dataset

print("\n\n\n Target dataset: \n\n",targets.isnull())

# nomissing values in target dataset

Ambient sensor dataset: 

     Sensor 1  Sensor 2  Sensor 3  Sensor 4
0      False     False     False     False
1      False     False     False     False
2      False     False     False     False
3      False     False     False     False
4      False     False     False     False
..       ...       ...       ...       ...
40     False     False     False     False
41     False     False     False     False
42     False     False     False     False
43     False     False     False     False
44     False     False     False     False

[8080 rows x 4 columns]



 Target dataset: 

      label
1    False
2    False
3    False
4    False
5    False
..     ...
212  False
213  False
214  False
215  False
216  False

[216 rows x 1 columns]


In [7]:
print(data.dtypes," \n\n", targets.dtypes)


Sensor 1    float64
Sensor 2    float64
Sensor 3    float64
Sensor 4    float64
dtype: object  

 label    int64
dtype: object


Statistical Description of sequences 

In [9]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
pd.Series(len_sequences).describe()

count    216.000000
mean      37.407407
std       14.948306
min       19.000000
25%       25.000000
50%       33.500000
75%       47.750000
max      103.000000
dtype: float64

Finding the 80th quartile for padding


In [10]:
Q = pd.Series(len_sequences).quantile(0.8)
print(" 80th Quartile is: ", Q)

 80th Quartile is:  54.0


**Padding:**

In [11]:
#Padding the sequence with the values in last row to max length 103
to_pad = 103
new_seq = []

for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
    #to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    to_concat = np.repeat([0,0,0,0],n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#truncate the sequence to length 54 based on 80th quartile
from keras.preprocessing import sequence
seq_len = 54
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

Using TensorFlow backend.


In [12]:
final_seq.shape

(216, 54, 4)

***Splitting Training and Testing Data***

> 80-20



In [13]:
train_ambient = [final_seq[i] for i in range(1,172)]
test_ambient = [final_seq[i] for i in range(173,215)]

train_ambient = np.array(train_ambient)
test_ambient = np.array(test_ambient)

train_target = [targets.label[i] for i in range(1,172)]
test_target = [targets.label[i] for i in range(173,215)]

train_target = np.array(train_target)
train_target = (train_target+1)/2

test_target = np.array(test_target)
test_target = (test_target+1)/2

print("**** Length of split data ****",
      "\n\n Ambient Training Data: ",len(train_ambient),
      "\n Ambient Testing Data: ", len(test_ambient),
      "\n Target Training Data: ", len(train_target),
      "\n Target Testing Data: ",len(test_target))

**** Length of split data **** 

 Ambient Training Data:  171 
 Ambient Testing Data:  42 
 Target Training Data:  171 
 Target Testing Data:  42


**Building Model:**

In [0]:
#### Base Model

def build_model():
  adam = Adam(lr=0.001)

  model = Sequential()
  model.add(Input(shape=(int(seq_len), 4)))
  #model.add(LSTM(units=16, return_sequences=True))
  model.add(LSTM(units=8, return_sequences=True))
  model.add(LSTM(units=4))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

  print(model.summary())

  test_preds = model.predict_classes(test_ambient)

  return model, test_preds

Testing base Model

In [37]:

base_model, test_preds = build_model()

#chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
train_1 = base_model.fit(train_ambient,
    train_target,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    validation_data=(test_ambient,test_target)
)

from sklearn.metrics import accuracy_score
#test_preds = model.predict_classes(test_ambient)
acc = accuracy_score(test_target, test_preds)
print("\n\n *********************************************\n\n")
print("Accuracy for the base model is: ", acc)
print("\n\n*********************************************")


Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_33 (LSTM)               (None, 54, 8)             416       
_________________________________________________________________
lstm_34 (LSTM)               (None, 4)                 208       
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 5         
Total params: 629
Trainable params: 629
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
3/3 [==============================] - 1s 234ms/step - loss: 0.7218 - accuracy: 0.4632 - val_loss: 0.7174 - val_accuracy: 0.6857
Epoch 2/100
3/3 [==============================] - 0s 37ms/step - loss: 0.7144 - accuracy: 0.4191 - val_loss: 0.7216 - val_accuracy: 0.5714
Epoch 3/100
3/3 [==============================] - 0s 36ms/step - loss: 0.7155 - accuracy: 0.4265 -

10-fold cross validation Function.....

In [0]:
def evaluate_model(ambient_train, target_train, ambient_test, target_test, mod, test_preds, k=10):
  kf = StratifiedKFold(n_splits=k)
  train_accuracy = []
  validation_accuracy = []
  train_f1 = []
  validation_f1 = []
  classes = ['0','1']
  count = 0
  for train, test in kf.split(ambient_train, target_train):
    count = count + 1
    # model.fit
    mod.fit( ambient_train[train], target_train[train], validation_split=0.2, epochs=100, batch_size=64, verbose=0)
    
   
    # calculating accuracy, f1 score, recall, confuion matrix

    # train

    train_preds = mod.predict(ambient_train[train]).T[0]
    

    class_pred = np.int32(np.array([1.0 if p > 0.5 else 0.0 for p in train_preds]))

    class_true = np.int32(target_train[train])

    train_precision, train_recall, train_f1_score, _ = precision_recall_fscore_support(class_true, class_pred)

    train_f1.append((train_precision, train_recall, train_f1_score))
    t_acc = accuracy_score(class_true, class_pred)
    train_accuracy.append(t_acc)
    
    
    # validation

    validation_preds = mod.predict(ambient_train[test]).T[0]
    

    class_pred = np.int32(np.array([1.0 if pr > 0.5 else 0.0 for pr in validation_preds]))


    class_true = np.int32(target_train[test])

    validation_precision, validation_recall, validation_f1_score, _ = precision_recall_fscore_support(class_true, class_pred)
    
    validation_f1.append((validation_precision, validation_recall, validation_f1_score))
    v_acc = accuracy_score(class_true, class_pred)
    validation_accuracy.append(v_acc)

    # K-Fold K counter:

    print("************** ",count, " Fold ******************")
    
    # classification report :

    cr = classification_report(class_true, class_pred, target_names= classes)
    print("\n Classification Report: \n", cr)

    # loss function, Accuracy :

    #ev = mod.evaluate(ambient_test, target_test, verbose = 2)
    #print("\n\n Loss: ", ev[0], "  Accuracy: ", ev[1])
    print("\n\n Training accuracy: ", t_acc)
    print("\n\n validation accuracy: ", v_acc)

    # Confusion Matrix :

    cfm = confusion_matrix(class_true, class_pred)
    print(cfm,"\n\n\n\n")

  # Mean Classification Scores of 10 Folds:

  print("##################### Total Mean Scores #####################\n\n")

  print(  "  Training Accuracy........", np.mean(train_accuracy))
  print("\n  Validation Accuracy......", np.mean(validation_accuracy))
  print("\n")
  print("\n  Training Precision.......", np.nan_to_num(np.mean([prf[0] for prf in train_f1])))
  print("\n  Training Recall..........", np.mean([prf[1] for prf in train_f1]))
  print("\n  Training f1 Score........", np.mean([prf[2] for prf in train_f1]))
  print("\n")
  print("\n  Validation Precision.....", np.mean([prf[0] for prf in validation_f1]))
  print("\n  Validation Recall........", np.mean([prf[1] for prf in validation_f1]))
  print("\n  Validation f1 Score......", np.mean([prf[2] for prf in validation_f1]))

  print("************************************************************\n\n")

  # classification report :

  cr1 = classification_report(class_true, class_pred, target_names= classes)
  print("\n Classification Report: \n", cr1)

  # loss function, Accuracy :

  ev1 = mod.evaluate(ambient_test, target_test, verbose = 2)
  print("\n\n Loss: ", ev1[0], "  Accuracy: ", ev1[1])

  # Confusion Matrix :

  cfm1 = confusion_matrix(class_true, class_pred)
  print("\nConfusion Matrix: \n\n",cfm1)


sending base model for cross validation

In [41]:
base_model, test_preds = build_model()
print("\n\n*******************************************************************************************************************************\n\n Evaluating Model")
evaluate_model(train_ambient, train_target, test_ambient, test_target, base_model,test_preds)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_37 (LSTM)               (None, 54, 8)             416       
_________________________________________________________________
lstm_38 (LSTM)               (None, 4)                 208       
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 5         
Total params: 629
Trainable params: 629
Non-trainable params: 0
_________________________________________________________________
None


*******************************************************************************************************************************

 Evaluating Model
**************  1  Fold ******************

 Classification Report: 
               precision    recall  f1-score   support

           0       0.56      1.00      0.71        10
           1       0.00      0.00      0.00         

**Optimized Model 1**

I have added two layers of LSTM having units 512 with return sequence true and 128. fitted this model with 200 epoch, validation split as 0.1 and batch size 128.

In [0]:
def build_model_1():
  adam = Adam(lr=0.001)

  model = Sequential()
  model.add(Input(shape=(int(seq_len), 4)))
  model.add(LSTM(units=512, return_sequences=True))
  model.add(LSTM(units=128))
  #model.add(LSTM(units=4))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

  print(model.summary())

  test_preds = model.predict_classes(test_ambient)

  return model, test_preds

Testing Model 1

In [57]:
model_1, test_preds = build_model_1()

train_2 = model_1.fit(train_ambient,
    train_target,
    validation_split=0.1,
    epochs=100,
    batch_size=128,
    validation_data=(test_ambient,test_target)
)

from sklearn.metrics import accuracy_score
#test_preds = model.predict_classes(test_ambient)
acc = accuracy_score(test_target, test_preds)
print("\n\n *********************************************\n\n")
print("Accuracy for the base model is: ", acc)
print("\n\n*********************************************")

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_65 (LSTM)               (None, 54, 512)           1058816   
_________________________________________________________________
lstm_66 (LSTM)               (None, 128)               328192    
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 129       
Total params: 1,387,137
Trainable params: 1,387,137
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
2/2 [==============================] - 1s 614ms/step - loss: 0.6959 - accuracy: 0.4967 - val_loss: 0.6939 - val_accuracy: 0.5000
Epoch 2/100
2/2 [==============================] - 1s 328ms/step - loss: 0.6491 - accuracy: 0.5752 - val_loss: 0.6096 - val_accuracy: 0.7778
Epoch 3/100
2/2 [==============================] - 1s 325ms/step - loss: 0.5917 - accu

In [44]:
model_1, test_preds = build_model_1()
print("\n\n*******************************************************************************************************************************\n\n Evaluating Model")
evaluate_model(train_ambient, train_target, test_ambient, test_target, model_1,test_preds)

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_39 (LSTM)               (None, 54, 512)           1058816   
_________________________________________________________________
lstm_40 (LSTM)               (None, 128)               328192    
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 129       
Total params: 1,387,137
Trainable params: 1,387,137
Non-trainable params: 0
_________________________________________________________________
None


*******************************************************************************************************************************

 Evaluating Model
**************  1  Fold ******************

 Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1

**Optimizing Model 2**

Now to optimize the model, I will add 2 dropout layers to prevent any over-fitting of data. Adding a dropout layer will reduce the interdependent learning amongst the neurons by regularizing neural network. Results were as following 76.19% accuracy without cross validation and 88.43% accuracy with 10-fold cross validation. 

In [0]:
def build_model_2():
  adam = Adam(lr=0.001)
  #admx = Adagrad(lr=0.002)

  model = Sequential()
  model.add(Input(shape=(int(seq_len), 4)))
  model.add(LSTM(units=32, return_sequences=True))
  model.add(Dropout(0.5))
  model.add(LSTM(units=8))
  model.add(Dropout(0.3))
  #model.add(LSTM(units=4))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

  print(model.summary())

  test_preds = model.predict_classes(test_ambient)

  return model, test_preds

In [61]:
model_2, test_preds = build_model_2()

train_2 = model_2.fit(train_ambient,
    train_target,
    validation_split=0.1,
    epochs=200,
    batch_size=64,
    validation_data=(test_ambient,test_target)
)

from sklearn.metrics import accuracy_score
#test_preds = model.predict_classes(test_ambient)
acc = accuracy_score(test_target, test_preds)
print("\n\n *********************************************\n\n")
print("Accuracy for the base model is: ", acc)
print("\n\n*********************************************")

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_71 (LSTM)               (None, 54, 32)            4736      
_________________________________________________________________
dropout_4 (Dropout)          (None, 54, 32)            0         
_________________________________________________________________
lstm_72 (LSTM)               (None, 8)                 1312      
_________________________________________________________________
dropout_5 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 9         
Total params: 6,057
Trainable params: 6,057
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
3/3 [==============================] - 1s 229ms/step - loss: 0.7013 - accuracy: 0.3725 - val_loss: 0

In [62]:
model_2, test_preds = build_model_2()
print("\n\n*******************************************************************************************************************************\n\n Evaluating Model")
evaluate_model(train_ambient, train_target, test_ambient, test_target, model_2,test_preds)

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_73 (LSTM)               (None, 54, 32)            4736      
_________________________________________________________________
dropout_6 (Dropout)          (None, 54, 32)            0         
_________________________________________________________________
lstm_74 (LSTM)               (None, 8)                 1312      
_________________________________________________________________
dropout_7 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 9         
Total params: 6,057
Trainable params: 6,057
Non-trainable params: 0
_________________________________________________________________
None


**************************************************************************************************************

**Optimizing model 3**

For optimizing further, I have tried to change the learning rate of the optimizer ‘adam’ from: 0.001 to 0.010. added another dense layer to the model and this time surprisingly I got better results. Accuracy results without cross validation gave me 90% accuracy and after implementing 10-fold cross validation on model 3, I got 94.92 (Approx 95%) accuracy.

In [0]:
def build_model_3():
  adam = Adam(lr=0.010)
  #admx = Adagrad(lr=0.002)

  model = Sequential()
  model.add(Input(shape=(int(seq_len), 4)))
  model.add(LSTM(units=16, return_sequences=True))
  model.add(Dropout(0.5))
  model.add(LSTM(units=8))
  model.add(Dense(32))
  model.add(Dropout(0.3))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

  print(model.summary())

  test_preds = model.predict_classes(test_ambient)

  return model, test_preds

In [70]:
model_3, test_preds = build_model_3()

train_3 = model_3.fit(train_ambient,
    train_target,
    validation_split=0.1,
    epochs=200,
    batch_size=64,
    validation_data=(test_ambient,test_target)
)

from sklearn.metrics import accuracy_score
#test_preds = model.predict_classes(test_ambient)
acc = accuracy_score(test_target, test_preds)
print("\n\n *********************************************\n\n")
print("Accuracy for the base model is: ", acc)
print("\n\n*********************************************")

Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_87 (LSTM)               (None, 54, 16)            1344      
_________________________________________________________________
dropout_20 (Dropout)         (None, 54, 16)            0         
_________________________________________________________________
lstm_88 (LSTM)               (None, 8)                 800       
_________________________________________________________________
dense_49 (Dense)             (None, 32)                288       
_________________________________________________________________
dropout_21 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 1)                 33        
Total params: 2,465
Trainable params: 2,465
Non-trainable params: 0
___________________________________________________

In [71]:
model_3, test_preds = build_model_3()
print("\n\n*******************************************************************************************************************************\n\n Evaluating Model")
evaluate_model(train_ambient, train_target, test_ambient, test_target, model_3,test_preds)

Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_89 (LSTM)               (None, 54, 16)            1344      
_________________________________________________________________
dropout_22 (Dropout)         (None, 54, 16)            0         
_________________________________________________________________
lstm_90 (LSTM)               (None, 8)                 800       
_________________________________________________________________
dense_51 (Dense)             (None, 32)                288       
_________________________________________________________________
dropout_23 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_52 (Dense)             (None, 1)                 33        
Total params: 2,465
Trainable params: 2,465
Non-trainable params: 0
___________________________________________________